In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urdu-stop-word/stopword.txt
/kaggle/input/urdu-news-data/description.csv
/kaggle/input/urdu-news-data/headline.csv


In [2]:
# Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import re
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


In [4]:
# Load the CSV file
data = pd.read_csv('/kaggle/input/urdu-news-data/description.csv',engine='python',encoding='utf-8')

data = data.dropna()
# Load data from CSV
# descriptions = data['Description'].tolist()
# categories = data['Category'].tolist()

In [5]:
stop_word_list = []
with open("/kaggle/input/urdu-stop-word/stopword.txt") as fin:
    for i in fin:
        stop_word_list.append(i.strip())
fin.close()

pattern = r'[a-zA-Z0-9!?:;"(,)]'
data['Description'] = data['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_word_list]))
data['Description'] = data['Description'].apply(lambda x: re.sub(pattern,'',x))

In [7]:
descriptions = data['Description'].tolist()
categories = data['Category'].tolist()

In [9]:
# Define maximum sequence length
MAX_SEQUENCE_LENGTH = 1000

In [38]:
# Tokenize text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(descriptions)
sequences = tokenizer.texts_to_sequences(descriptions)

In [39]:
# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Encoding categories
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(categories)
num_classes = len(label_encoder.classes_)

In [40]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(padded_sequences, integer_encoded_labels, test_size=0.2)


In [30]:
input_dim=len(tokenizer.word_index) + 1
print(input_dim)

90191


In [41]:
# Model definition
model = tf.keras.Sequential([
    Embedding(5000, 128, input_length=MAX_SEQUENCE_LENGTH),
    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [42]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [43]:
# Train the model
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))


Epoch 1/10
4431/4431 [==============================] - 64s 14ms/step - loss: 1.2295 - accuracy: 0.5346 - val_loss: 1.1142 - val_accuracy: 0.5814
Epoch 2/10
4431/4431 [==============================] - 28s 6ms/step - loss: 1.0520 - accuracy: 0.5994 - val_loss: 1.1038 - val_accuracy: 0.5775
Epoch 3/10
4431/4431 [==============================] - 27s 6ms/step - loss: 0.9660 - accuracy: 0.6212 - val_loss: 1.1213 - val_accuracy: 0.5722
Epoch 4/10
4431/4431 [==============================] - 27s 6ms/step - loss: 0.8835 - accuracy: 0.6430 - val_loss: 1.1827 - val_accuracy: 0.5562
Epoch 5/10
4431/4431 [==============================] - 26s 6ms/step - loss: 0.8081 - accuracy: 0.6599 - val_loss: 1.2566 - val_accuracy: 0.5469
Epoch 6/10
4431/4431 [==============================] - 26s 6ms/step - loss: 0.7448 - accuracy: 0.6767 - val_loss: 1.3795 - val_accuracy: 0.5348
Epoch 7/10
4431/4431 [==============================] - 26s 6ms/step - loss: 0.6950 - accuracy: 0.6898 - val_loss: 1.4683 - val_a

In [44]:
predictions = model.predict(test_data)

1108/1108 [==============================] - 2s 2ms/step


In [46]:
# Generate classification report
y_pred = np.argmax(predictions, axis=1)
print("CNN Report")
print(classification_report(test_labels, y_pred))

CNN Report
              precision    recall  f1-score   support

           0       0.03      0.02      0.02       164
           1       0.64      0.72      0.68      6187
           2       0.36      0.32      0.34      6336
           3       0.49      0.49      0.49      5099
           4       0.37      0.26      0.31      6561
           5       0.74      0.77      0.76      5021
           6       0.41      0.52      0.46      6081

    accuracy                           0.50     35449
   macro avg       0.43      0.44      0.44     35449
weighted avg       0.49      0.50      0.49     35449

